# Introduction to Optical Coherence Tomography (OCT)

Welcome to the first lesson on OCT! In this notebook, we'll explore the fundamental concepts that make OCT possible.

## What is OCT?

Optical Coherence Tomography (OCT) is a non-invasive imaging technique that uses light to capture high-resolution, cross-sectional images of biological tissues. Think of it as "ultrasound with light" - but with much better resolution!

### Key Features:
- **Resolution**: 1-15 micrometers (much better than ultrasound)
- **Penetration depth**: 1-2 mm in biological tissue
- **Non-invasive**: Uses safe near-infrared light
- **Real-time**: Can capture images in real-time

## Learning Objectives

By the end of this notebook, you will understand:
1. The principle of low-coherence interferometry
2. The Michelson interferometer setup
3. How OCT creates depth-resolved images
4. Basic OCT terminology

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert

# Set up plotting parameters
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")

## 1. Low-Coherence Interferometry

OCT is based on **low-coherence interferometry**. Let's break down what this means:

### Coherence

Coherence is a measure of how well a light wave maintains a consistent phase relationship over time and space.

- **High coherence** (e.g., laser): Light waves stay "in sync" for a long time
- **Low coherence** (e.g., LED): Light waves quickly lose their phase relationship

### Why Low Coherence?

Low-coherence light only produces interference when the path lengths are nearly equal. This is the key to depth resolution in OCT!

**Coherence length**: The distance over which light maintains coherence

$$l_c = \frac{2 \ln 2}{\pi} \frac{\lambda_0^2}{\Delta\lambda}$$

where:
- $\lambda_0$ is the center wavelength
- $\Delta\lambda$ is the bandwidth

In [ ]:
def calculate_coherence_length(wavelength_nm, bandwidth_nm):
    """
    Calculate the coherence length of a light source.

    Parameters:
    - wavelength_nm: Center wavelength in nanometers
    - bandwidth_nm: Bandwidth in nanometers

    Returns:
    - Coherence length in micrometers
    """
    wavelength_um = wavelength_nm / 1000  # Convert to micrometers
    bandwidth_um = bandwidth_nm / 1000

    coherence_length = (2 * np.log(2) / np.pi) * (wavelength_um**2 / bandwidth_um)
    return coherence_length

# Example: Common OCT light source
wavelength = 840  # nm (near-infrared)
bandwidth = 50    # nm (broadband)

lc = calculate_coherence_length(wavelength, bandwidth)
print(f"Center wavelength: {wavelength} nm")
print(f"Bandwidth: {bandwidth} nm")
print(f"Coherence length: {lc:.2f} μm")
print(f"\nThis coherence length determines the axial resolution of the OCT system!")

### Interactive: Explore Coherence Length

Let's see how bandwidth affects coherence length:

In [ ]:
# Explore different bandwidths
wavelengths = [840, 1300]  # Common OCT wavelengths
bandwidths = np.linspace(10, 100, 100)  # Range of bandwidths

plt.figure(figsize=(12, 5))

for wavelength in wavelengths:
    coherence_lengths = [calculate_coherence_length(wavelength, bw) for bw in bandwidths]
    plt.plot(bandwidths, coherence_lengths, label=f'λ = {wavelength} nm', linewidth=2)

plt.xlabel('Bandwidth (nm)', fontsize=12)
plt.ylabel('Coherence Length (μm)', fontsize=12)
plt.title('Coherence Length vs. Bandwidth', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

print("Key insight: Larger bandwidth → Shorter coherence length → Better axial resolution!")

## 2. The Michelson Interferometer

OCT uses a Michelson interferometer as its core optical setup:

```
                    Sample Arm
                         ↓
Light Source → Beam Splitter → Sample (Tissue)
                         ↓
                   Reference Arm
                         ↓
                   Reference Mirror
                         ↓
                      Detector
```

### How it works:

1. **Light is split**: A beam splitter divides light into two paths
2. **Sample arm**: Light travels to the tissue and reflects back
3. **Reference arm**: Light travels to a mirror and reflects back
4. **Interference**: The two beams recombine at the detector
5. **Detection**: Interference only occurs when path lengths match (within coherence length)

### Interference Condition

Interference occurs when:

$$|z_{sample} - z_{reference}| < l_c$$

This is how we get depth information!

In [ ]:
# Simulate interference signal
def interference_signal(z_reference, z_sample, coherence_length, wavelength=840e-9):
    """
    Simulate the interference signal from a Michelson interferometer.

    Parameters:
    - z_reference: Reference arm length (meters)
    - z_sample: Sample position (meters)
    - coherence_length: Coherence length (meters)
    - wavelength: Center wavelength (meters)

    Returns:
    - Interference signal intensity
    """
    # Path length difference
    delta_z = z_sample - z_reference

    # Envelope (Gaussian) due to low coherence
    envelope = np.exp(-2 * (delta_z / coherence_length)**2)

    # Carrier (interference fringes)
    carrier = np.cos(4 * np.pi * delta_z / wavelength)

    # Combined signal
    signal = envelope * carrier

    return signal

# Simulation parameters
wavelength = 840e-9  # 840 nm in meters
coherence_length = 7e-6  # 7 μm in meters
z_reference = 100e-6  # 100 μm reference position

# Sample positions to scan
z_sample_positions = np.linspace(80e-6, 120e-6, 1000)  # 80-120 μm range

# Calculate interference signal
signals = [interference_signal(z_reference, z, coherence_length, wavelength)
           for z in z_sample_positions]

# Plot
plt.figure(figsize=(12, 6))
plt.plot(z_sample_positions * 1e6, signals, linewidth=1.5)
plt.axvline(z_reference * 1e6, color='r', linestyle='--', label='Reference position', linewidth=2)
plt.xlabel('Sample Position (μm)', fontsize=12)
plt.ylabel('Interference Signal', fontsize=12)
plt.title('Interference Signal vs. Sample Position', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

print("Notice: Strong interference only occurs near the reference position!")
print("This is the key to depth-resolved imaging in OCT.")

## 3. Creating Depth-Resolved Images (A-scans)

An **A-scan** (axial scan) is a 1D depth profile of the tissue.

To create an A-scan:
1. Scan the reference mirror (or analyze the spectrum)
2. Detect interference at each position
3. The envelope of the interference signal reveals tissue structure

Let's simulate a simple tissue with multiple layers:

In [ ]:
# Simulate a multi-layer tissue sample
def tissue_reflectivity(z_positions, layer_positions, layer_reflectivities):
    """
    Create a tissue model with multiple reflecting layers.

    Parameters:
    - z_positions: Array of depth positions
    - layer_positions: List of layer depths
    - layer_reflectivities: List of reflectivity values (0-1)

    Returns:
    - Reflectivity profile
    """
    reflectivity = np.zeros_like(z_positions)

    for layer_z, layer_r in zip(layer_positions, layer_reflectivities):
        # Create a narrow peak at each layer position
        reflectivity += layer_r * np.exp(-((z_positions - layer_z) / 1e-6)**2)

    return reflectivity

# Define tissue structure (e.g., skin layers)
layer_positions = [100e-6, 150e-6, 200e-6, 280e-6]  # μm
layer_names = ['Surface', 'Epidermis', 'Dermis boundary', 'Deep structure']
layer_reflectivities = [0.8, 0.5, 0.6, 0.3]

# Depth positions for simulation (these are now treated as reference arm positions)
# We need a finer sampling for simulating the interference fringes
z_reference_positions = np.linspace(50e-6, 350e-6, 2000)

# Simulate OCT A-scan (interference signals)
# For each reference position, sum the interference from all layers
oct_interference_signal = np.zeros_like(z_reference_positions)

# We need to consider interference from each layer at each reference position
for i, z_ref in enumerate(z_reference_positions):
    for layer_z, layer_r in zip(layer_positions, layer_reflectivities):
        oct_interference_signal[i] += layer_r * interference_signal(z_ref, layer_z, coherence_length, wavelength)

# To get the A-scan (depth profile), we need to find the envelope of the interference signal
# This is typically done using the Hilbert transform in signal processing
oct_a_scan = np.abs(hilbert(oct_interference_signal))


# Plot results
fig, axes = plt.subplots(2, 1, figsize=(12, 8)) # Changed to 2 rows, 1 column for vertical stack

# True tissue structure
z_positions_tissue = np.linspace(50e-6, 350e-6, 1000)
true_tissue_reflectivity = tissue_reflectivity(z_positions_tissue, layer_positions, layer_reflectivities)

axes[0].plot(z_positions_tissue * 1e6, true_tissue_reflectivity, 'b', linewidth=2)
axes[0].set_xlabel('Depth (μm)', fontsize=12)
axes[0].set_ylabel('Reflectivity', fontsize=12)
axes[0].set_title('True Tissue Structure', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
for pos, name in zip(layer_positions, layer_names):
    axes[0].axvline(pos * 1e6, color='r', linestyle='--', alpha=0.5)
axes[0].set_xlim([50, 350])


# Simulated OCT Signal and A-scan (Envelope of the interference signal)
axes[1].plot(z_reference_positions * 1e6, oct_interference_signal, 'b', linewidth=1, label='Raw Interference Signal')
axes[1].plot(z_reference_positions * 1e6, oct_a_scan, 'g', linewidth=2, label='OCT A-scan (Envelope)')
axes[1].set_xlabel('Depth (μm)', fontsize=12) # X-axis now represents depth inferred from reference position
axes[1].set_ylabel('Signal Intensity', fontsize=12)
axes[1].set_title('Simulated OCT Signal and A-scan', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Add vertical lines at the layer positions for comparison
for pos in layer_positions:
     axes[1].axvline(pos * 1e6, color='r', linestyle='--', alpha=0.5)
axes[1].set_xlim([50, 350])
axes[1].legend()


plt.tight_layout()
plt.show()

print("The simulated OCT A-scan (envelope of the interference signal) now shows peaks corresponding to the tissue layers!")
print("This is how the depth profile of the tissue is extracted in OCT.")

## 4. Key OCT Terminology

Let's define some important terms:

### A-scan (Axial scan)
- 1D depth profile at a single lateral position
- Shows reflectivity vs. depth

### B-scan (Brightness scan)
- 2D cross-sectional image
- Created by combining multiple A-scans

### Axial Resolution
- Resolution in the depth direction
- Determined by coherence length: $\Delta z \approx \frac{l_c}{2}$

### Lateral Resolution
- Resolution in the transverse direction
- Determined by focusing optics: $\Delta x \approx \frac{4\lambda f}{\pi D}$
- Where f is focal length, D is beam diameter

### Imaging Depth
- Maximum depth that can be imaged
- Limited by light scattering (typically 1-2 mm in tissue)

### Sensitivity
- Minimum detectable reflectivity
- Typically 90-110 dB in modern systems

In [ ]:
# Calculate axial and lateral resolution
def calculate_axial_resolution(wavelength_nm, bandwidth_nm):
    """Calculate axial resolution (approximately half the coherence length)."""
    lc = calculate_coherence_length(wavelength_nm, bandwidth_nm)
    return lc / 2

def calculate_lateral_resolution(wavelength_nm, focal_length_mm, beam_diameter_mm):
    """Calculate lateral resolution based on focusing optics."""
    wavelength_um = wavelength_nm / 1000
    resolution = (4 * wavelength_um * focal_length_mm) / (np.pi * beam_diameter_mm)
    return resolution

# Example system parameters
wavelength = 840  # nm
bandwidth = 50    # nm
focal_length = 20  # mm
beam_diameter = 2  # mm

axial_res = calculate_axial_resolution(wavelength, bandwidth)
lateral_res = calculate_lateral_resolution(wavelength, focal_length, beam_diameter)

print("=" * 50)
print("OCT System Specifications")
print("=" * 50)
print(f"Center wavelength: {wavelength} nm")
print(f"Bandwidth: {bandwidth} nm")
print(f"Focal length: {focal_length} mm")
print(f"Beam diameter: {beam_diameter} mm")
print("\n" + "=" * 50)
print("Calculated Performance")
print("=" * 50)
print(f"Axial resolution: {axial_res:.2f} μm")
print(f"Lateral resolution: {lateral_res:.2f} μm")
print("\nNote: These are theoretical values. Actual performance depends on")
print("      many factors including sample scattering and system noise.")

## Summary

In this introductory notebook, we've learned:

1. ✅ **OCT basics**: Non-invasive imaging with light, achieving micrometer-scale resolution

2. ✅ **Low-coherence interferometry**: The key enabling technology that provides depth selectivity

3. ✅ **Michelson interferometer**: The optical setup that splits, reflects, and recombines light

4. ✅ **A-scans**: How we extract depth information from interference signals

5. ✅ **Resolution**: The factors that determine image quality (bandwidth for axial, optics for lateral)

## Next Steps

In the next notebook, we'll dive deeper into:
- Different types of interferometry
- Coherence properties in detail
- How to simulate realistic interference patterns

## Exercises

Try these on your own:

1. Modify the bandwidth in the first example and observe how it affects coherence length
2. Add more layers to the tissue simulation and see how they appear in the A-scan
3. Calculate the specifications needed for a 5 μm axial resolution at 1300 nm wavelength

---

**Continue to Notebook 2: Interferometry Basics →**